<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/1010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
N-gram => Tokenize, LM
1. Tokenizer => Collocation(이웃한 토큰) 패턴을 찾을 수 있음
  => 어절(단어), 음절, 형태소, ... => 단일 단어(토큰)의 형태
  => 단일 토큰들의 N개의 쌍 => 고려대+학교, 고려+대학교, 고려대학교, 고+려대학교, ..
                          한성대+학교, 외국어대학교, ...
                          ; 이름 + 학교, 대학교 => 2,3토큰
                          ; [이름 + 학교, 대학교] => 1토큰
2. LM
  => P(A,B,C,D,...,Z) 못구함; 데이터X, P=0
  => [1st, 2nd, 3rd, ...] Markov Assumption
     Bigram; P(A,B,C,D,...,Z)=P(Z|A,...,Y)P(A,...,Y)
                            ~=P(Z|X,Y)P(Y|X)...
                             => P(Z,Y)/P(Y) = freq(Z,Y)/freq(Y)
            A+B+C, A+B+D, A+B+E, ...
            A+B + [?]
            학+교+에(P)/에서(P*)/를/...
            P => freq(A,B,C?, D?, E?)/ferq(A,B)
3. Perplexity => 응집력
   (ㅠP(i|0,...,i-1))**(1/len(i))
     P(A,B,C,D) => P(D|A,B,C)P(A,B,C)
    ----------- = P(D|A,B,C)
      P(A,B,C)
   ; P(A,B,C)/P(A,B) * P(A,B)/P(A)
   => P(A,B,C,D)/P(A)
      = freq(A,B,C,D)/freq(A)
   P(고려대학교|고), P(고려대학교|고려)
                  => 고려대 (8), 고려학원 (2), ..., 고려대학교(90)
                     8/100     2/100           90/100
    알잘~... => ?
N-gram 띄어쓰기
Cohesion Score 분절찾기
내일;Entropy + BytePairEncoding
내일;Normalization + Stopwords
import re
from os import listdir

path = 'naver'

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))
from collections import Counter

gram1 = list()
gram2 = list()
gram3 = list()
gram4 = list()

for f in fileids('naver'):
    with open(f, 'r', encoding='utf8') as fp:
        doc = re.sub(r'\s+', ' ', fp.read())
        gram1.extend(ngram(doc, 1))
        gram2.extend(ngram(doc, 2))
        gram3.extend(ngram(doc, 3))
        gram4.extend(ngram(doc, 4))

gram1 = Counter(gram1)
gram2 = Counter(gram2)
gram3 = Counter(gram3)
gram4 = Counter(gram4)
len(gram1), len(gram2), len(gram3), len(gram4)
(1186, 17075, 46925, 77077)
'코스피가 이스라엘-팔레스타인 전쟁에 따른 국제정세 불안 속에서도 장초반 1% 넘게 오르고 있다.'
data = re.sub(r'\s', '', '코스피가 이스라엘-팔레스타인 전쟁에 따른 국제정세 불안 속에서도 장초반 1% 넘게 오르고 있다.')
# 다음글자가 ' '인지 보는 함수
#      => 확률적으로 가장 큰 다음글자를 return
# N-gram(bigram;(A,B*)/(A*), trigram;(A,B,C*)/(A,B*))
def spacing(s):
    gram = {1:gram1, 2:gram2, 3:gram3, 4:gram4}
    given = gram[len(s)]
    candi = gram[len(s)+1]

    c = dict(filter(lambda r:re.match(s, r[0]), candi.items()))

    maxkey = ' '
    if len(c) > 0:
        maxkey = max(c, key=c.get)
#     print(c)
#     print(maxkey, c[maxkey])

    return maxkey
spacing('학교')[-1]
' '
N = 2
rst = ''
data = '한 번 저지른 일은 되돌릴 수 없다며 강하게 방류 중단을 요구하고 있습니다.'.replace(' ', '')
for i in range(len(data)):
    if i < N:
        if N > 1:
            key = rst[-N:] if len(rst)>1 else data[:i+1]
            if spacing(key)[-1] == ' ':
                rst += ' '
        rst += data[i]
    else:
        if spacing(rst[-N:])[-1] == ' ':
            rst += ' '
        rst += data[i]
rst
' 한번 저지른 일은 되돌릴 수 없다며 강하게 방류 중단을 요구하고 있습니다.'
# Cohesion Score => Perplexity
image.png

# 형태소+형태소+형태소 => 어절
# 고려대는, 고려대에서는, 고려대와, 고려대로, ...
# Perplexity(고려대에서는) = P(려|고)*P(대|고려)*P(에|고려대)*...
#                      => (P(고려)/P(고))*(P(고려대)/P(고려))*...
#                      => P(고려대에서는)/P(고) > P(고려대)/P(고)
from nltk.tokenize import word_tokenize, regexp_tokenize

word = list()
for f in fileids('naver'):
    with open(f, 'r', encoding='utf8') as fp:
        word.extend(regexp_tokenize(re.sub(r'\s+', ' ', fp.read()),
                                    r'\b\w+\b'))
word = Counter(word)
# word로부터 s로 시작하는 모든 빈도의 합 return
def find(s):
    return sum(dict(filter(lambda r:r[0].startswith(s),
                    word.items())
                   ).values())
key = '말했다'
before = 0.0
for i in range(1,len(key)):
    p = find(key[0:i+1])/find(key[0])
    print(key[:i+1], find(key[0:i+1])/find(key[0]))
    print('자름' if before > p else '놔둠')
    before = p
말했 0.5392156862745098
놔둠
말했다 0.5294117647058824
자름
# Branching Entropy
entropy: -sum(p log p=.5)
|
|      *(불확실성)
|   *    *
-[*]--|---[*]--1*--------2

말했다, 말했고, 말했는데, 말했었고, ...
P => P(다|말했) P(고|말했) P(는|말했) P(었|말했)
def findkey(s):
    return list(
                set(
                    list(
                    map(lambda k:k[:len(s)+1],
                        list(dict(filter(lambda r:r[0].startswith(s),
                                     word.items())
                             ).keys()
                        )))
                )
           )
from math import log

key = '아르헨티나'
before = 0.0
for i in range(len(key)):
    total = 0.0
    for k in findkey(key[:i+1]):
        p = find(k)/find(key[:i+1])
    #     print(k, -p*log(p))
        total += -p*log(p)
    print(key[:i+1], total, '자름' if before < total else '냅둠')
    before = total
아 3.010214705057504 자름
아르 0.18490739916777568 냅둠
아르헨 0.0 냅둠
아르헨티 0.0 냅둠
아르헨티나 1.0287719079404922 자름
findkey('아르헨티나')
['아르헨티나', '아르헨티나가', '아르헨티나로', '아르헨티나의', '아르헨티나뿐', '아르헨티나에']
raw = '''
low low low low low lowest lowest newer newer
newer newer newer newer wider wider wider
'''

# 단어 = 어절
data = dict()
for k, v in Counter(raw.split()).items():
    data[' '.join(tuple(k))+' </w>'] = v
data
{'l o w </w>': 5,
 'l o w e s t </w>': 2,
 'n e w e r </w>': 6,
 'w i d e r </w>': 3}
def findpair(d, n=2):
    rst = dict()

    for k, v in d.items():
#         'l o w </w>' => ['l', 'o', 'w', '</w>']
        k = k.split()
        for i in range(len(k)-(n-1)):
            key = ' '.join(k[i:i+n])
            if key in rst:
                rst[key] += v
            else:
                rst[key] = v
    return rst
keys = findpair(data)
best = max(keys, key=keys.get)
rst = dict()
for k, v in data.items():
    rst[re.sub(best, best.replace(' ', ''), k)] = v
for _ in range(5):
    keys = findpair(data)
    best = max(keys, key=keys.get)

    rst = dict()
    for k, v in data.items():
        rst[re.sub(best, best.replace(' ', ''), k)] = v

    data = rst
    print(best, rst)
e r {'l o w </w>': 5, 'l o w e s t </w>': 2, 'n e w er </w>': 6, 'w i d er </w>': 3}
er </w> {'l o w </w>': 5, 'l o w e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
l o {'lo w </w>': 5, 'lo w e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
lo w {'low </w>': 5, 'low e s t </w>': 2, 'n e w er</w>': 6, 'w i d er</w>': 3}
n e {'low </w>': 5, 'low e s t </w>': 2, 'ne w er</w>': 6, 'w i d er</w>': 3}
[k.split() for k in data]
[['low', '</w>'],
 ['low', 'e', 's', 't', '</w>'],
 ['ne', 'w', 'er</w>'],
 ['w', 'i', 'd', 'er</w>']]
data = dict()
for k, v in word.items():
    data[' '.join(tuple(k))] = v

for _ in range(100):
    keys = findpair(data)
    best = max(keys, key=keys.get)

    rst = dict()
    for k, v in data.items():
        rst[re.sub(best, best.replace(' ', ''), k)] = v

    data = rst
    print(best.replace(' ', ''))
에서
했다
으로
있다
니다
하고
하는
00
이다
습니다
한다
20
있는
이라
까지
지난
한국
지만
서울
후보
대한
들이
기술
10
에는
레이
됐다
부터
202
일본
이어
면서
하기
5일
시장
판매
기자
연구
통령
미국
대통령
우리
적으로
전기
중국
다는
적인
영화
뉴스
위해
었다
통해
정부
플레이
경제
관련
이라고
사진
기업
에게
라고
하게
예술
디스
디스플레이
후보자
마트
달러
다고
부산
시간
전자
개발
말했다
이번
하지
민주
19
것으로
된다
밝혔
제공
보다
들의
의원
스마트
들어
하며
이날
시스
산업
는데
최근
기를
하다
대표
국제
세계
에도
사람
Tokeninzing** => 전처리 시작
1. 문장 분리
2. tokens 분리 => Feature Extraction/Selection
   => 구두점 토큰화, 정규식, 트위터, ...
      형태소, 품사, 명사 => 사전
      stem(어간)ming, lema(표제어,어근)tization
      Eng. Porter's s/es/ed/..,
      N-gram
      Kor. Perplexity, Entropy, BPE
      WPM, SPM
3. Normalization => Filtering/Selection
from nltk.corpus import stopwords
stop = stopwords.open(
    stopwords.fileids()[stopwords.fileids().index('english')]).read()
from string import punctuation

s = 'To be or not to be.'
rst = []
for w in re.sub(f'[{re.escape(punctuation)}]', '', s.lower()).split():
    if w not in stop:
        rst.append(w)
    else:
        print(f'skipped:{w}')
' '.join(rst)
skipped:to
skipped:be
skipped:or
skipped:not
skipped:to
skipped:be
''